In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162990 sha256=4ab6ef7f86b4816ace4706909aeb1b183b8df5a196628262f86713126b390df4
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import surprise

In [3]:
ratings = pd.read_csv("u.data",sep='\t',names = ['uid','iid','rating','ts'])
ratings.drop('ts', axis=1, inplace=True)
ratings.head()

,uid,iid,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [4]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))

Ratings range between 1 and 5


Converting the data into **surprise** format:

In [5]:
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

**SVD** Tuning with different hyper-parameters

In [6]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

param_grid = {'n_epochs': np.arange(5,50,10),
              'lr_all':np.linspace(0.001,1,5),
              'reg_all': np.linspace(0.01,0.8,5),
              'n_factors':[50,100]}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.SVD, param_grid, measures=['rmse', 'mae'], n_jobs=-1,
                  cv=kfold, joblib_verbose=3)

Running the Grid Search CV

In [7]:
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed: 23.2min
[Parallel(n_jobs=-1)]: Done 1250 out of 1250 | elapsed: 26.7min finished


**Best Score**

In [8]:
print(gs.best_score['rmse'])

0.9482558403165307


**Best Param**

In [9]:
print(gs.best_params['rmse'])

{'n_epochs': 45, 'lr_all': 0.001, 'reg_all': 0.01, 'n_factors': 50}
